In [1]:
# IMPORT LIBRARY
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
# MEMUAT LOAD DATA
df = pd.read_csv('Online_Retail.csv', encoding='latin-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [3]:
# MENAMPILKAN INFO DATAFRAME
df.info(),
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


(541909, 8)

In [4]:
# CEK MISSING VALUE
missing_value = df.isnull().sum()

In [5]:
# MENGHAPUS BARIS YANG MEMILIKI MISSING VALUE
df = df.dropna()

# MENGHAPUS DUPLICATE
df = df.drop_duplicates()

# MENGUBAH INVOICE DATE MENJADI DATETIME
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

C:\Users\62822\AppData\Local\Temp\ipykernel_34712\1717358457.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


In [6]:
# CLUSTERING DATA (KMEANS)
df_sample = df.sample(250000, random_state=42)
cluster_df = df_sample[['Quantity', 'UnitPrice']]
scaler = StandardScaler()
cluster_scaled = scaler.fit_transform(cluster_df)

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(cluster_scaled)
cluster_df['Cluster'] = kmeans_labels

C:\Users\62822\AppData\Local\Temp\ipykernel_34712\2714065644.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_df['Cluster'] = kmeans_labels


In [ ]:
# EVALUASI MODEL CLUSTERING
sil_score = silhouette_score(cluster_scaled, kmeans_labels)
print(f'silhouette_score(sample): {sil_score * 100:.2f}%')

In [ ]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
threshold = df['TotalPrice'].median()
df['HighPurchase'] = df['TotalPrice'].apply(lambda x: 1 if x > threshold else 0)

In [ ]:
# CLASSIFICATION DATA
classification_df = df[['Quantity', 'UnitPrice', 'TotalPrice', 'HighPurchase']]. sample(250000, random_state=42)

In [ ]:
# MEMISAHKAN DATA TRAINING DAN TESTING
X = classification_df[['Quantity', 'UnitPrice', 'TotalPrice']].copy()
y = classification_df['HighPurchase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6817
           1       1.00      1.00      1.00      6683

    accuracy                           1.00     13500
   macro avg       1.00      1.00      1.00     13500
weighted avg       1.00      1.00      1.00     13500



In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[['Quantity','UnitPrice', 'TotalPrice']])

pca = PCA(n_components=2)
X_pca = pca.fit_transform(scaled_data)

# Tampilkan varians yang dijelaskan oleh setiap komponen
print(pca.explained_variance_ratio_)

# Ubah array hasil PCA ke DataFrame
X_pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])

# Visualisasi
plt.figure(figsize=(8, 6))
plt.scatter(X_pca_df['PC1'], X_pca_df['PC2'])
plt.title('Visualisasi PCA (2D)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True)
plt.show()

In [ ]:
# AKURASI DATA
accuracy = model.score(X_test, y_test)
print(f'Akurasi Model: {accuracy * 100:.2f}%')

Akurasi Model: 99.98%
